In [3]:
# import necessary libraries
import numpy as np
import pandas as pd
import matplotlib as plt

# available datasets
recipes = pd.read_csv("/data/foodboost/recipes.csv")
ingredients = pd.read_csv("/data/foodboost/ingredients.csv")
nutritions = pd.read_csv("/data/foodboost/nutritions.csv")
tags = pd.read_csv("/data/foodboost/tags.csv")

# clean up data
del recipes['Unnamed: 0']
del ingredients['Unnamed: 0']
del nutritions['Unnamed: 0']
del tags['Unnamed: 0']
recipes.title = recipes.title.str.replace('&nbsp;', ' ')
ingredients.recipe = ingredients.recipe.str.replace('&nbsp;', ' ')
nutritions.recipe = nutritions.recipe.str.replace('&nbsp;', ' ')
tags.recipe = tags.recipe.str.replace('&nbsp;', ' ')
recipes.title = recipes.title.str.replace('&amp;', 'en')
ingredients.recipe = ingredients.recipe.str.replace('&amp;', 'en')
nutritions.recipe = nutritions.recipe.str.replace('&amp;', 'en')
tags.recipe = tags.recipe.str.replace('&amp;', 'en')
tags.tag = tags.tag.str.replace('&amp;', 'en')

In [4]:
# explore recipes dataset
print(recipes.head(), "\n")
print(recipes.info(), "\n")
print(recipes.describe(), "\n")
print(recipes.shape, "\n")

                                title  persons  time  calories  stars  \
0       Kruidnoten met choco-discodip        4    25       260      0   
1             Kruidnoten in marsepein        4    25       265      0   
2            Kruidnoten met chocodips        4    25       335      0   
3  Pepernotentaart met marsepeinstrik       10    30       560      0   
4                           Perencake       12    10       265      0   

                                                 url  \
0  https://www.ah.nl/allerhande/recept/R-R1195893...   
1  https://www.ah.nl/allerhande/recept/R-R1195892...   
2  https://www.ah.nl/allerhande/recept/R-R1195891...   
3  https://www.ah.nl/allerhande/recept/R-R1195887...   
4  https://www.ah.nl/allerhande/recept/R-R1195790...   

                                               image  
0  https://static.ah.nl/static/recepten/img_RAM_P...  
1  https://static.ah.nl/static/recepten/img_RAM_P...  
2  https://static.ah.nl/static/recepten/img_RAM_P...  
3  h

In [5]:
# explore nutritions dataset
print(nutritions.head(), "\n")
print(nutritions.info(), "\n")
print(nutritions.describe(), "\n")
print(nutritions.shape, "\n")

                          recipe        nutrition     value
0  Kruidnoten met choco-discodip          energie  260 kcal
1  Kruidnoten met choco-discodip     koolhydraten      34 g
2  Kruidnoten met choco-discodip  waarvan suikers      22 g
3  Kruidnoten met choco-discodip          natrium    200 mg
4  Kruidnoten met choco-discodip            eiwit       3 g 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58765 entries, 0 to 58764
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   recipe     58765 non-null  object
 1   nutrition  58765 non-null  object
 2   value      58765 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB
None 

                                          recipe nutrition  value
count                                      58765     58765  58765
unique                                      8705         8   1192
top     Broccolistamppot met noten en geitenkaas   energie    1 g
freq                    

In [6]:
# explore ingredients dataset
print(ingredients.head(), "\n")
print(ingredients.info(), "\n")
print(ingredients.describe(), "\n")
print(ingredients.shape, "\n")

                          recipe            ingredient  quantity unit
0  Kruidnoten met choco-discodip         melkchocolade     100.0    g
1  Kruidnoten met choco-discodip            kruidnoten     100.0    g
2  Kruidnoten met choco-discodip              discodip       2.0   el
3        Kruidnoten in marsepein      blanke marsepein     150.0    g
4        Kruidnoten in marsepein  ongezouten roomboter      15.0    g 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71809 entries, 0 to 71808
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   recipe      71809 non-null  object 
 1   ingredient  71809 non-null  object 
 2   quantity    71809 non-null  float64
 3   unit        57509 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.2+ MB
None 

           quantity
count  71809.000000
mean     106.405130
std      173.488264
min        0.050000
25%        2.000000
50%        5.000000
75%      150.000000
max  

In [7]:
# explore tags dataset
print(tags.head(), "\n")
print(tags.info(), "\n")
print(tags.describe(), "\n")
print(tags.shape, "\n")

                          recipe               tag
0  Kruidnoten met choco-discodip          hollands
1  Kruidnoten met choco-discodip             gebak
2  Kruidnoten met choco-discodip             gebak
3  Kruidnoten met choco-discodip       sinterklaas
4  Kruidnoten met choco-discodip  sinterklaasavond 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46040 entries, 0 to 46039
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   recipe  46040 non-null  object
 1   tag     46040 non-null  object
dtypes: object(2)
memory usage: 719.5+ KB
None 

                recipe           tag
count            46040         46040
unique            8704           118
top     Tomatentulpjes  hoofdgerecht
freq                18          4595 

(46040, 2) 



In [8]:
# select only vegetarian main dishes
hoofd_recipes = tags.loc[tags['tag'] == 'hoofdgerecht']
hoofd_recipes = pd.merge(hoofd_recipes, tags, on='recipe', how='left')
veg_recipes = hoofd_recipes.loc[hoofd_recipes['tag_y'] == 'vegetarisch']

In [9]:
# merge veg_recipes and ingredients
veg_ingredients = pd.merge(veg_recipes, ingredients, how='left')
del veg_ingredients['tag_x']
del veg_ingredients['tag_y']
del veg_ingredients['quantity']
del veg_ingredients['unit']



# create recipe:[ingredients] dictionary
veg_dict = {}
test = veg_ingredients.groupby('recipe')['ingredient'].apply(list)
for i in test.keys():
    veg_dict[i] = test[i]
print(veg_dict["'Foeyonghai' met bami"])

['knoflook', 'sesamolie', 'tomato frito', 'gembersiroop', 'sojasaus', 'nasi-bamigroente', 'babymais uit blik', 'mienestje', 'middelgroot ei', 'kraanwater', 'Saitaku zwart sesamzaad geroosterd in strooier']


In [10]:
# merge veg_recipes and nutritions
veg_nutritions = pd.merge(veg_recipes, nutritions, on='recipe', how='left')
del veg_nutritions['tag_x']
del veg_nutritions['tag_y']
print(veg_nutritions)

                                            recipe          nutrition  \
0     Pasta pesto met kipstuckjes en tomatensalade            energie   
1     Pasta pesto met kipstuckjes en tomatensalade       koolhydraten   
2     Pasta pesto met kipstuckjes en tomatensalade    waarvan suikers   
3     Pasta pesto met kipstuckjes en tomatensalade            natrium   
4     Pasta pesto met kipstuckjes en tomatensalade              eiwit   
...                                            ...                ...   
4626                              Ovenpasta met ei            natrium   
4627                              Ovenpasta met ei              eiwit   
4628                              Ovenpasta met ei                vet   
4629                              Ovenpasta met ei  waarvan verzadigd   
4630                              Ovenpasta met ei             vezels   

         value  
0     770 kcal  
1         63 g  
2          9 g  
3       640 mg  
4         31 g  
...        ...  
4626

In [11]:
# get calorie values
veg_calories = veg_nutritions[veg_nutritions['nutrition'] == 'energie']
del veg_calories['nutrition']
print(veg_calories.head())

                                               recipe     value
0        Pasta pesto met kipstuckjes en tomatensalade  770 kcal
8           Hutspot met vegetarische rookworst en jus  535 kcal
16  Zoete aardappelstamppot met spruiten en vega g...  595 kcal
24   'Kapsalon' met vegetarische kipshoarma en salade  505 kcal
32  Gebakken krieltjes met broccoli en vegetarisch...  535 kcal
